In [1]:
import os
os.chdir("../") # resets notebook directory to repository root folder (DO ONLY ONCE!)
import polars as pl
import pandas as pd
import pyarrow.parquet as pypq
import textwrap
from pathlib import Path
import time

from tqdm.auto import tqdm
tqdm.pandas()

In [2]:
import sys
# Add utils directory in the list of directories to look for packages to import
sys.path.insert(0, os.path.join(os.getcwd(),'utils'))
from read_parquet import *

Using PyArrow strings!


In [3]:
works_by_topic_parquet_folder = "data/works_by_topic_parquet/"
works2year_by_topic_parquet_folder = "data/works2year_by_topic_parquet/"
os.makedirs(works2year_by_topic_parquet_folder, exist_ok=True)

In [4]:
topics = [topic[:-8] for topic in os.listdir(works_by_topic_parquet_folder)]

In [5]:
len(topics)

4516

In [6]:
topic = topics[0]
works_df = read_parquet(works_by_topic_parquet_folder+topic, columns = ["id", "date"])


Reading 'T14064' from 'data/works_by_topic_parquet/T14064.parquet' using engine='pyarrow'
Read 36,130 rows from 'T14064' in 0.37 sec.
Converting dtypes took 0.00 sec. Size before: 0.00GB, after: 0.00GB


,id,date
0,W7126435799,2026-02-01
1,W7126372280,2026-02-01
2,W7126447841,2026-01-31


In [7]:
def flattenList(l):
    return [item for sublist in l for item in sublist]

def generate_work2year_df(parquet_file_path):
    works_df = read_parquet(parquet_file_path, columns = ["id", "date", "authors"], quiet = True)
    works_df['year'] = works_df['date'].apply(lambda x: int(x[:4]))
    works_df["n_authors"] = works_df["authors"].apply(lambda x : len(x.split(";")))
    return works_df[["id", "year", "n_authors"]]

def create_work2year_df(origin_works_parquet_file_path, destination_topic_parquet_file_path, compression='brotli', do_peek = True, do_print = True):
    works_df = generate_work2year_df(origin_works_parquet_file_path)
    works_df.to_parquet(destination_topic_parquet_file_path, compression=compression)
    if do_print:
        print(f"Successfully generated {destination_topic_parquet_file_path} using {origin_works_parquet_file_path}.")
    if do_peek:
        if do_print:
            print("Here's a peek.")
        peek_parquet(destination_topic_parquet_file_path)

In [8]:
for topic in tqdm(topics):
    origin_works_parquet_file_path = works_by_topic_parquet_folder+topic+".parquet"
    destination_topic_parquet_file_path = works2year_by_topic_parquet_folder+topic+".parquet"
    create_work2year_df(origin_works_parquet_file_path, destination_topic_parquet_file_path, do_peek = False, do_print = False)

  0%|          | 0/4516 [00:00<?, ?it/s]

In [9]:
peek_parquet(destination_topic_parquet_file_path)

Name: 'T10091'
Path: 'data/works2year_by_topic_parquet/T10091.parquet'
Files: 1
Rows: 160,055
Schema:
    id: large_string
    year: int64
    n_authors: int64
    -- schema metadata --
    pandas: '{"index_columns": [{"kind": "range", "name": null, "start": 0, "' + 587
5 random rows:


,id,year,n_authors
0,W7083686573,2030,1
1,W6963238420,2028,1
2,W6929729901,2028,3
3,W6925566458,2028,1
4,W7089278898,2026,1
